In [2]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

import pandas, seaborn, numpy, collections, numpy, pickle
%matplotlib inline
from matplotlib import pyplot
import topiary
import mhctools
import mhcflurry
from varcode.common import memoize
pandas.set_option('display.max_columns', None)
import matplotlib


def print_full(x):
    pandas.set_option('display.max_rows', len(x))
    print(x)
    pandas.reset_option('display.max_rows')

matplotlib.rc("savefig", dpi=800)
#matplotlib.rc("savefig", dpi=72)
matplotlib.rc('text', usetex=False)
#reload(c)

%matplotlib inline
%config InlineBackend.figure_format = 'png'



/Users/tim/miniconda3/envs/py3k/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
Using Theano backend.


In [3]:
# Write out mutation counts
mutations = pandas.read_csv("../data/derived/annotated_mutations.with_mhc_binders.csv.bz2",
                            converters={'binding_peptides': lambda x: eval(x) if x else {}})
mutations = mutations.ix[mutations.confident]

/Users/tim/miniconda3/envs/py3k/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
mutations.shape

(1070404, 34)

In [5]:
sources = pandas.read_csv("../data/derived/sources.full.csv", index_col="source_id")
sources["source_id"] = sources.index
treated_paired_donors = set(d for d in sources.donor
                              if len(sources.ix[sources.donor == d].treated.unique()) > 1 )
treated_paired_donors
sources["treated_paired"] = sources.donor.isin(treated_paired_donors)
sources

,RNA_ID,RNA biospecimen,DNA_id,DNA_biospecimen,specimen_type_description,CollectionPoint,SpecimenType,RNA data file,DNA_sample_string,icgc_sample_id,project_code_x,submitted_sample_id,icgc_specimen_id,submitted_specimen_id_x,icgc_donor_id_x,submitted_donor_id_x,analyzed_sample_interval,percentage_cellularity_x,level_of_cellularity_x,study,project_code_y,study_specimen_involved_in,submitted_specimen_id_y,icgc_donor_id_y,submitted_donor_id_y,specimen_type,specimen_type_other,specimen_interval,specimen_donor_treatment_type,specimen_donor_treatment_type_other,specimen_processing,specimen_processing_other,specimen_storage,specimen_storage_other,tumour_confirmed,specimen_biobank,specimen_biobank_id,specimen_available,tumour_histological_type,tumour_grading_system,tumour_grade,tumour_grade_supplemental,tumour_stage_system,tumour_stage,tumour_stage_supplemental,digital_image_of_stained_section,percentage_cellularity_y,level_of_cellularity_y,cohort,donor,tissue_type,timepoint,treated,metastasis,interval_days,bam_path_tumor_dna,bam_path_tumor_rna,treated_paired,timepoint_paired,source_id
source_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AOCS-001-1-7,ICGCDBDE20130916001,AOCS-001-2-0,AOCS_001_ICGC_DBPC_20130205_002,AOCS-001-1-7,primary tumour,Primary,Tumour,130906_D81P8DQ1_0153_C2704ACXX.nopd.AOCS_001_I...,ICGCDBPC20130205002,SA505244,OV-AU,AOCS-001-1-7,SP101515,AOCS-001-1,DO46325,AOCS-001,1895.0,73.0,61-80%,PCAWG,OV-AU,PCAWG,AOCS-001-1,DO46325,AOCS-001,Primary tumour - solid tissue,NaN,0,no treatment,NaN,fresh,NaN,"frozen, vapor phase",NaN,no,NaN,NaN,NaN,8441/3,Silverberg,3.0,NaN,FIGO,III,NaN,NaN,73.0,61-80%,AOCS,AOCS-001,solid,primary,False,False,0,/demeter/scratch/datasets/ega-tim/EGAZ00001018...,/demeter/scratch/datasets/ega-tim/_EGAZ0000101...,False,False,AOCS-001-1-7
AOCS-004-1-5,ICGCDBDE20130916003,AOCS-004-2-9,AOCS_004_ICGC_DBPC_20130205_004,AOCS-004-1-5,primary tumour,Primary,Tumour,130906_D81P8DQ1_0157_C270DACXX.nopd.AOCS_004_I...,ICGCDBPC20130205004,SA505270,OV-AU,AOCS-004-1-5,SP101519,AOCS-004-1,DO46327,AOCS-004,1350.0,77.0,61-80%,PCAWG,OV-AU,PCAWG,AOCS-004-1,DO46327,AOCS-004,Primary tumour - solid tissue,NaN,0,no treatment,NaN,fresh,NaN,"frozen, vapor phase",NaN,no,NaN,NaN,NaN,8441/3,Silverberg,3.0,NaN,FIGO,III,NaN,NaN,77.0,61-80%,AOCS,AOCS-004,solid,primary,False,False,0,/demeter/scratch/datasets/ega-tim/_EGAZ0000101...,/demeter/scratch/datasets/ega-tim/_EGAZ0000101...,False,False,AOCS-004-1-5
AOCS-005-1-8,ICGCDBDE20130916004,AOCS-005-2-1,AOCS_005_ICGC_DBPC_20130205_006,AOCS-005-1-8,primary tumour,Primary,Tumour,130906_D81P8DQ1_0157_C270DACXX.nopd.AOCS_005_I...,ICGCDBPC20130205006,SA505282,OV-AU,AOCS-005-1-8,SP101521,AOCS-005-1,DO46328,AOCS-005,2128.0,90.0,>81%,PCAWG,OV-AU,PCAWG,AOCS-005-1,DO46328,AOCS-005,Primary tumour - solid tissue,NaN,0,no treatment,NaN,fresh,NaN,"frozen, vapor phase",NaN,no,NaN,NaN,NaN,8441/3,Silverberg,3.0,NaN,FIGO,III,NaN,NaN,90.0,>81%,AOCS,AOCS-005,solid,primary,False,False,0,/demeter/scratch/datasets/ega-tim/_EGAZ0000101...,/demeter/scratch/datasets/ega-tim/_EGAZ0000101...,False,False,AOCS-005-1-8
AOCS-034-1-0,ICGCDBLG2010050402TR,AOCS-034-2-4,AOCS_034_ICGC_DBPC_20130205_009,AOCS-034-1-0,primary tumour,Primary,Tumour,131206_EXTERN_0045_AC2KJBACXX.nopd.AOCS_034_IC...,ICGCDBPC20130205009,SA505297,OV-AU,AOCS-034-1-0,SP101523,AOCS-034-1,DO46329,AOCS-034,1953.0,84.0,>81%,PCAWG,OV-AU,PCAWG,AOCS-034-1,DO46329,AOCS-034,Primary tumour - solid tissue,NaN,0,no treatment,NaN,fresh,NaN,"frozen, vapor phase",NaN,no,NaN,NaN,NaN,8140/3,Silverberg,3.0,NaN,FIGO,III,NaN,NaN,84.0,>81%,AOCS,AOCS-034,solid,primary,False,False,0,/demeter/scratch/datasets/ega-tim/_EGAZ0000101...,/demeter/scratch/datasets/ega-tim/_EGAZ0000101...,True,True,AOCS-034-1-0
AOCS-034-3-8,ICGCDBDE20131122024,AOCS-034-4-1,AOCS_034_ICGC_DBPC_20130205_007,AOCS-034-3-8,recurrent ascitic fluid,Recurrence,Ascites,131206_EXTERN_0045_AC2KJBACXX.nopd.AOCS_034_IC...,ICGCDBPC20130205007,SA505303,OV-AU,AOCS-034-3-8,SP101524,AOCS-034-13,DO4

In [6]:
muts_new = mutations.ix[[
    (sources.ix[s].treated) for s in mutations.source_id
]].ix[
    mutations.snv &
    mutations.donor.isin(treated_paired_donors) &
    mutations.unique_to_treated
].groupby(["source_id", "context_mutation_3p5p"]).interbase_start.count().fillna(0).unstack().fillna(0)
muts_new["kind"] = "new"
muts_new

context_mutation_3p5p,A(C>A)A,A(C>A)C,A(C>A)G,A(C>A)T,A(C>G)A,A(C>G)C,A(C>G)G,A(C>G)T,A(C>T)A,A(C>T)C,A(C>T)G,A(C>T)T,A(T>A)A,A(T>A)C,A(T>A)G,A(T>A)T,A(T>C)A,A(T>C)C,A(T>C)G,A(T>C)T,A(T>G)A,A(T>G)C,A(T>G)G,A(T>G)T,C(C>A)A,C(C>A)C,C(C>A)G,C(C>A)T,C(C>G)A,C(C>G)C,C(C>G)G,C(C>G)T,C(C>T)A,C(C>T)C,C(C>T)G,C(C>T)T,C(T>A)A,C(T>A)C,C(T>A)G,C(T>A)T,C(T>C)A,C(T>C)C,C(T>C)G,C(T>C)T,C(T>G)A,C(T>G)C,C(T>G)G,C(T>G)T,G(C>A)A,G(C>A)C,G(C>A)G,G(C>A)T,G(C>G)A,G(C>G)C,G(C>G)G,G(C>G)T,G(C>T)A,G(C>T)C,G(C>T)G,G(C>T)T,G(T>A)A,G(T>A)C,G(T>A)G,G(T>A)T,G(T>C)A,G(T>C)C,G(T>C)G,G(T>C)T,G(T>G)A,G(T>G)C,G(T>G)G,G(T>G)T,T(C>A)A,T(C>A)C,T(C>A)G,T(C>A)T,T(C>G)A,T(C>G)C,T(C>G)G,T(C>G)T,T(C>T)A,T(C>T)C,T(C>T)G,T(C>T)T,T(T>A)A,T(T>A)C,T(T>A)G,T(T>A)T,T(T>C)A,T(T>C)C,T(T>C)G,T(T>C)T,T(T>G)A,T(T>G)C,T(T>G)G,T(T>G)T,kind
source_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AOCS-034-3-8,69,83,7,64,72,44,16,84,79,33,55,71,41,30,21,58,76,25,38,42,23,5,39,35,84,61,10,119,43,36,12,81,70,279,37,260,46,44,55,118,36,42,34,58,18,16,37,29,66,73,5,43,45,44,5,72,80,73,43,67,13,16,22,32,35,31,24,41,11,11,22,19,95,75,10,99,326,82,18,485,259,96,40,172,37,33,25,53,49,39,14,37,32,18,37,68,new
AOCS-064-3-3,145,154,10,176,127,61,32,139,171,81,65,132,79,74,79,127,107,69,72,143,49,23,66,68,120,105,13,124,78,58,25,79,81,82,58,194,127,103,127,219,56,118,85,256,34,44,74,187,150,151,9,90,62,156,6,115,123,144,41,105,54,49,41,103,51,90,41,85,17,49,49,80,77,98,10,113,100,92,18,182,85,66,36,120,84,72,42,185,96,111,37,129,50,37,70,159,new
AOCS-065-3-6,25,23,3,30,24,15,5,42,33,18,23,26,17,8,14,32,26,9,11,27,9,4,13,8,28,21,3,27,23,12,5,28,30,43,19,48,18,21,13,28,6,13,15,21,7,2,13,23,26,15,1,19,9,17,1,27,28,29,22,25,9,14,15,10,14,12,1,17,10,4,11,12,28,27,2,25,33,17,1,38,24,27,15,25,7,18,10,21,21,10,6,24,18,7,12,20,new
AOCS-086-3-2,97,125,6,78,83,42,23,92,80,56,83,93,44,28,40,58,83,34,62,93,31,11,33,29,89,78,10,128,62,49,21,86,125,361,49,310,112,101,129,187,57,62,64,109,23,20,44,53,115,126,13,79,46,115,9,115,118,107,64,104,15,29,24,38,47,53,38,47,15,13,29,24,73,71,8,100,120,76,11,204,106,79,33,103,71,39,24,77,59,60,31,71,34,14,46,77,new
AOCS-088-3-8,201,228,17,204,134,78,27,126,188,98,95,198,78,61,82,144,122,84,73,142,43,18,72,49,174,172,21,199,89,73,19,116,121,137,67,231,108,142,136,232,62,135,58,85,24,30,73,73,126,113,15,147,60,71,11,91,131,89,77,130,62,73,69,139,54,50,42,62,35,16,63,52,208,154,21,260,327,152,32,513,412,188,45,315,96,93,61,206,90,99,45,98,45,38,93,87,new
AOCS-091-3-0,138,165,15,120,163,61,21,120,158,73,121,126,63,67,50,94,136,71,109,123,31,17,55,37,138,126,12,170,81,43,12,128,79,163,90,190,131,118,175,220,61,83,67,113,31,28,57,51,93,145,15,87,63,90,7,125,156,103,90,102,45,47,48,77,65,72,53,87,20,22,56,39,110,99,9,124,121,92,10,211,105,86,50,105,95,73,50,137,98,84,49,99,51,30,55,78,new
AOCS-092-3-3,74,161,2,88,44,17,5,42,53,24,52,56,40,51,28,44,42,27,30,49,22,10,20,34,109,76,21,200,22,24,10,63,149,534,45,513,312,171,265,358,53,43,48,105,21,19,37,30,81,154,6,68,18,110,3,64,107,105,48,57,22,27,15,29,23,69,19,27,8,23,12,18,56,65,8,111,54,32,8,131,81,89,37,114,41,28,24,48,27,35,17,41,9,16,29,42,new
AOCS-093-3-6,56,80,4,55,48,24,13,41,49,34,29,56,27,12,38,40,66,34,44,53,10,8,21,20,37,48,4,65,30,46,15,55,56,127,26,128,83,58,91,80,34,41,42,57,11,14,33,25,53,69,8,32,22,59,7,51,69,53,24,48,15,16,29,22,26,37,21,28,7,8,14,10,47,37,7,64,89,59,6,129,100,53,17,74,30,16,19,45,42,34,21,46,24,18,25,42,new
AOCS-094-6-X,95,104,10,101,96,61,27,106,119,53,67,129,44,42,45,90,91,60,74,104,35,10,42,24,95,94,6,86,61,50,18,94,57,44,50,110,47,66,61,98,47,106,53,64,15,30,56,29,48,59,9,68,42,32,9,55,71,49,43,67,29,26,25,52,49,45,40,47,21,7,38,31,98,56,9,96,176,84,18,237,94,60,28,103,71,37,38,84,81,110,46,113,38,25,58,74,new


In [7]:
muts_all = mutations.ix[
    mutations.snv
].groupby(["source_id", "context_mutation_3p5p"]).interbase_start.count().unstack().T.fillna(0)
#muts_all.columns = ["%s__all" % (x.replace("/", "_")) for x in muts_all.columns]
muts_all = muts_all.T
muts_all["kind"] = "all"

print(muts_all.isnull().sum().sum())
muts_all


0


context_mutation_3p5p,A(C>A)A,A(C>A)C,A(C>A)G,A(C>A)T,A(C>G)A,A(C>G)C,A(C>G)G,A(C>G)T,A(C>T)A,A(C>T)C,A(C>T)G,A(C>T)T,A(T>A)A,A(T>A)C,A(T>A)G,A(T>A)T,A(T>C)A,A(T>C)C,A(T>C)G,A(T>C)T,A(T>G)A,A(T>G)C,A(T>G)G,A(T>G)T,C(C>A)A,C(C>A)C,C(C>A)G,C(C>A)T,C(C>G)A,C(C>G)C,C(C>G)G,C(C>G)T,C(C>T)A,C(C>T)C,C(C>T)G,C(C>T)T,C(T>A)A,C(T>A)C,C(T>A)G,C(T>A)T,C(T>C)A,C(T>C)C,C(T>C)G,C(T>C)T,C(T>G)A,C(T>G)C,C(T>G)G,C(T>G)T,G(C>A)A,G(C>A)C,G(C>A)G,G(C>A)T,G(C>G)A,G(C>G)C,G(C>G)G,G(C>G)T,G(C>T)A,G(C>T)C,G(C>T)G,G(C>T)T,G(T>A)A,G(T>A)C,G(T>A)G,G(T>A)T,G(T>C)A,G(T>C)C,G(T>C)G,G(T>C)T,G(T>G)A,G(T>G)C,G(T>G)G,G(T>G)T,T(C>A)A,T(C>A)C,T(C>A)G,T(C>A)T,T(C>G)A,T(C>G)C,T(C>G)G,T(C>G)T,T(C>T)A,T(C>T)C,T(C>T)G,T(C>T)T,T(T>A)A,T(T>A)C,T(T>A)G,T(T>A)T,T(T>C)A,T(T>C)C,T(T>C)G,T(T>C)T,T(T>G)A,T(T>G)C,T(T>G)G,T(T>G)T,kind
source_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AOCS-001-1-7,120.0,83.0,15.0,91.0,84.0,48.0,12.0,79.0,92.0,53.0,122.0,102.0,30.0,32.0,44.0,59.0,124.0,48.0,80.0,120.0,24.0,15.0,32.0,30.0,98.0,85.0,13.0,80.0,50.0,38.0,12.0,72.0,72.0,74.0,73.0,109.0,44.0,58.0,71.0,66.0,35.0,69.0,47.0,42.0,16.0,16.0,25.0,30.0,63.0,69.0,8.0,56.0,42.0,42.0,5.0,39.0,88.0,75.0,82.0,67.0,29.0,19.0,42.0,54.0,54.0,29.0,47.0,67.0,4.0,4.0,33.0,27.0,75.0,93.0,19.0,102.0,148.0,86.0,12.0,231.0,158.0,108.0,63.0,121.0,51.0,51.0,27.0,91.0,67.0,47.0,37.0,81.0,22.0,20.0,35.0,62.0,all
AOCS-004-1-5,109.0,107.0,17.0,81.0,78.0,47.0,12.0,72.0,136.0,67.0,131.0,99.0,54.0,57.0,62.0,82.0,155.0,59.0,103.0,155.0,23.0,15.0,34.0,33.0,124.0,79.0,13.0,133.0,57.0,51.0,26.0,78.0,99.0,77.0,101.0,132.0,36.0,44.0,54.0,78.0,85.0,78.0,85.0,118.0,22.0,31.0,46.0,93.0,50.0,44.0,9.0,56.0,39.0,33.0,7.0,47.0,90.0,79.0,123.0,107.0,39.0,34.0,39.0,48.0,93.0,61.0,75.0,110.0,17.0,12.0,30.0,39.0,67.0,77.0,10.0,116.0,59.0,68.0,15.0,147.0,142.0,102.0,57.0,124.0,80.0,42.0,42.0,104.0,112.0,93.0,41.0,95.0,27.0,28.0,62.0,68.0,all
AOCS-005-1-8,107.0,65.0,6.0,60.0,55.0,38.0,18.0,53.0,120.0,55.0,167.0,101.0,48.0,26.0,32.0,71.0,124.0,57.0,98.0,125.0,32.0,16.0,37.0,53.0,56.0,44.0,9.0,45.0,43.0,32.0,14.0,56.0,84.0,59.0,143.0,83.0,23.0,34.0,32.0,52.0,43.0,51.0,45.0,71.0,22.0,18.0,31.0,59.0,66.0,42.0,10.0,53.0,25.0,18.0,10.0,31.0,76.0,66.0,133.0,62.0,24.0,11.0,13.0,31.0,48.0,25.0,33.0,50.0,24.0,10.0,43.0,35.0,85.0,83.0,21.0,117.0,147.0,73.0,11.0,251.0,206.0,102.0,91.0,178.0,61.0,14.0,19.0,65.0,58.0,50.0,37.0,78.0,36.0,20.0,44.0,83.0,all
AOCS-034-1-0,158.0,103.0,16.0,92.0,114.0,57.0,16.0,106.0,145.0,73.0,144.0,99.0,52.0,40.0,50.0,90.0,165.0,61.0,67.0,108.0,41.0,16.0,29.0,33.0,89.0,94.0,15.0,96.0,65.0,53.0,21.0,105.0,94.0,76.0,92.0,108.0,46.0,63.0,67.0,71.0,47.0,54.0,47.0,101.0,17.0,31.0,52.0,46.0,66.0,63.0,9.0,71.0,46.0,42.0,10.0,74.0,107.0,68.0,102.0,83.0,35.0,28.0,42.0,54.0,64.0,28.0,62.0,50.0,30.0,26.0,170.0,43.0,96.0,129.0,16.0,116.0,166.0,92.0,12.0,249.0,167.0,115.0,70.0,134.0,62.0,39.0,31.0,94.0,96.0,71.0,34.0,83.0,42.0,20.0,43.0,83.0,all
AOCS-034-3-8,234.0,193.0,27.0,157.0,187.0,97.0,30.0,184.0,234.0,108.0,193.0,164.0,87.0,65.0,71.0,142.0,253.0,82.0,103.0,148.0,66.0,21.0,65.0,67.0,176.0,162.0,24.0,230.0,109.0,93.0,32.0,191.0,167.0,408.0,132.0,412.0,92.0,104.0,122.0,196.0,89.0,95.0,89.0,158.0,37.0,48.0,89.0,69.0,138.0,163.0,15.0,114.0,93.0,97.0,15.0,157.0,185.0,152.0,156.0,155.0,49.0,45.0,68.0,89.0,88.0,62.0,82.0,86.0,40.0,34.0,239.0,63.0,205.0,212.0,25.0,219.0,506.0,180.0,28.0,773.0,462.0,212.0,121.0,317.0,103.0,68.0,56.0,151.0,137.0,106.0,46.0,126.0,74.0,38.0,79.0,160.0,all
AOCS-055-1-7,147.0,120.0,17.0,133.0,144.0,78.0,41.0,144.0,181.0,94.0,310.0,138.0,55.0,49.0,63.0,114.0,242.0,105.0,164.0,281.0,41.0,8.0,56.0,64.0,129.0,88.0,14.0,117.0,124.0,87.0,37.0,143.0,122.0,112.0,178.0,198.0,49.0,53.0,75.0,97.0,101.0,136.0,94.0,150.0,38.0,47.0,82.0,118.0,87.0,74.0,19.0,68.0,63.0,58.0,15.0,107.0,137.0,97.0,221.0,106.0,38.0,39.0,42.0,73.0,113.0,72.0,86.0,132.0,32.0,24.0,69.0,62.0,254.0,148.0,26.0,193.0,1586.0,372.0,94.0,1621.0,1028.0,331.0,225.0,594.0,89.0,50.0,50.0,124.0,134.0

In [8]:
# STATEMENT_UNIQUE_TO_TREATED_COUNT
print(mutations.ix[[
    (sources.ix[s].treated) for s in mutations.source_id
]].ix[
    mutations.snv &
    mutations.donor.isin(treated_paired_donors) &
    mutations.unique_to_treated
].shape)

print(mutations.ix[[
    (sources.ix[s].treated) for s in mutations.source_id
]].ix[
    mutations.snv &
    mutations.donor.isin(treated_paired_donors)
].shape)

(93986, 34)
(206766, 34)


In [11]:
93986.0 / 206766

0.45455248928740705

In [10]:
# To be loaded into R to use deconstructSigs package
muts_r = pandas.concat([muts_all, muts_new])

muts_r.columns = [x.replace("(", "[").replace(")", "]") for x in muts_r.columns]
muts_r["source_id"] = muts_r.index
muts_r = muts_r[["source_id"] + list(muts_r.columns)[:-1]]
muts_r.to_csv("../data/derived/mutation_contexts_counts.csv", index=False)
muts_r

,source_id,A[C>A]A,A[C>A]C,A[C>A]G,A[C>A]T,A[C>G]A,A[C>G]C,A[C>G]G,A[C>G]T,A[C>T]A,A[C>T]C,A[C>T]G,A[C>T]T,A[T>A]A,A[T>A]C,A[T>A]G,A[T>A]T,A[T>C]A,A[T>C]C,A[T>C]G,A[T>C]T,A[T>G]A,A[T>G]C,A[T>G]G,A[T>G]T,C[C>A]A,C[C>A]C,C[C>A]G,C[C>A]T,C[C>G]A,C[C>G]C,C[C>G]G,C[C>G]T,C[C>T]A,C[C>T]C,C[C>T]G,C[C>T]T,C[T>A]A,C[T>A]C,C[T>A]G,C[T>A]T,C[T>C]A,C[T>C]C,C[T>C]G,C[T>C]T,C[T>G]A,C[T>G]C,C[T>G]G,C[T>G]T,G[C>A]A,G[C>A]C,G[C>A]G,G[C>A]T,G[C>G]A,G[C>G]C,G[C>G]G,G[C>G]T,G[C>T]A,G[C>T]C,G[C>T]G,G[C>T]T,G[T>A]A,G[T>A]C,G[T>A]G,G[T>A]T,G[T>C]A,G[T>C]C,G[T>C]G,G[T>C]T,G[T>G]A,G[T>G]C,G[T>G]G,G[T>G]T,T[C>A]A,T[C>A]C,T[C>A]G,T[C>A]T,T[C>G]A,T[C>G]C,T[C>G]G,T[C>G]T,T[C>T]A,T[C>T]C,T[C>T]G,T[C>T]T,T[T>A]A,T[T>A]C,T[T>A]G,T[T>A]T,T[T>C]A,T[T>C]C,T[T>C]G,T[T>C]T,T[T>G]A,T[T>G]C,T[T>G]G,T[T>G]T,kind
source_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AOCS-001-1-7,AOCS-001-1-7,120.0,83.0,15.0,91.0,84.0,48.0,12.0,79.0,92.0,53.0,122.0,102.0,30.0,32.0,44.0,59.0,124.0,48.0,80.0,120.0,24.0,15.0,32.0,30.0,98.0,85.0,13.0,80.0,50.0,38.0,12.0,72.0,72.0,74.0,73.0,109.0,44.0,58.0,71.0,66.0,35.0,69.0,47.0,42.0,16.0,16.0,25.0,30.0,63.0,69.0,8.0,56.0,42.0,42.0,5.0,39.0,88.0,75.0,82.0,67.0,29.0,19.0,42.0,54.0,54.0,29.0,47.0,67.0,4.0,4.0,33.0,27.0,75.0,93.0,19.0,102.0,148.0,86.0,12.0,231.0,158.0,108.0,63.0,121.0,51.0,51.0,27.0,91.0,67.0,47.0,37.0,81.0,22.0,20.0,35.0,62.0,all
AOCS-004-1-5,AOCS-004-1-5,109.0,107.0,17.0,81.0,78.0,47.0,12.0,72.0,136.0,67.0,131.0,99.0,54.0,57.0,62.0,82.0,155.0,59.0,103.0,155.0,23.0,15.0,34.0,33.0,124.0,79.0,13.0,133.0,57.0,51.0,26.0,78.0,99.0,77.0,101.0,132.0,36.0,44.0,54.0,78.0,85.0,78.0,85.0,118.0,22.0,31.0,46.0,93.0,50.0,44.0,9.0,56.0,39.0,33.0,7.0,47.0,90.0,79.0,123.0,107.0,39.0,34.0,39.0,48.0,93.0,61.0,75.0,110.0,17.0,12.0,30.0,39.0,67.0,77.0,10.0,116.0,59.0,68.0,15.0,147.0,142.0,102.0,57.0,124.0,80.0,42.0,42.0,104.0,112.0,93.0,41.0,95.0,27.0,28.0,62.0,68.0,all
AOCS-005-1-8,AOCS-005-1-8,107.0,65.0,6.0,60.0,55.0,38.0,18.0,53.0,120.0,55.0,167.0,101.0,48.0,26.0,32.0,71.0,124.0,57.0,98.0,125.0,32.0,16.0,37.0,53.0,56.0,44.0,9.0,45.0,43.0,32.0,14.0,56.0,84.0,59.0,143.0,83.0,23.0,34.0,32.0,52.0,43.0,51.0,45.0,71.0,22.0,18.0,31.0,59.0,66.0,42.0,10.0,53.0,25.0,18.0,10.0,31.0,76.0,66.0,133.0,62.0,24.0,11.0,13.0,31.0,48.0,25.0,33.0,50.0,24.0,10.0,43.0,35.0,85.0,83.0,21.0,117.0,147.0,73.0,11.0,251.0,206.0,102.0,91.0,178.0,61.0,14.0,19.0,65.0,58.0,50.0,37.0,78.0,36.0,20.0,44.0,83.0,all
AOCS-034-1-0,AOCS-034-1-0,158.0,103.0,16.0,92.0,114.0,57.0,16.0,106.0,145.0,73.0,144.0,99.0,52.0,40.0,50.0,90.0,165.0,61.0,67.0,108.0,41.0,16.0,29.0,33.0,89.0,94.0,15.0,96.0,65.0,53.0,21.0,105.0,94.0,76.0,92.0,108.0,46.0,63.0,67.0,71.0,47.0,54.0,47.0,101.0,17.0,31.0,52.0,46.0,66.0,63.0,9.0,71.0,46.0,42.0,10.0,74.0,107.0,68.0,102.0,83.0,35.0,28.0,42.0,54.0,64.0,28.0,62.0,50.0,30.0,26.0,170.0,43.0,96.0,129.0,16.0,116.0,166.0,92.0,12.0,249.0,167.0,115.0,70.0,134.0,62.0,39.0,31.0,94.0,96.0,71.0,34.0,83.0,42.0,20.0,43.0,83.0,all
AOCS-034-3-8,AOCS-034-3-8,234.0,193.0,27.0,157.0,187.0,97.0,30.0,184.0,234.0,108.0,193.0,164.0,87.0,65.0,71.0,142.0,253.0,82.0,103.0,148.0,66.0,21.0,65.0,67.0,176.0,162.0,24.0,230.0,109.0,93.0,32.0,191.0,167.0,408.0,132.0,412.0,92.0,104.0,122.0,196.0,89.0,95.0,89.0,158.0,37.0,48.0,89.0,69.0,138.0,163.0,15.0,114.0,93.0,97.0,15.0,157.0,185.0,152.0,156.0,155.0,49.0,45.0,68.0,89.0,88.0,62.0,82.0,86.0,40.0,34.0,239.0,63.0,205.0,212.0,25.0,219.0,506.0,180.0,28.0,773.0,462.0,212.0,121.0,317.0,103.0,68.0,56.0,151.0,137.0,106.0,46.0,126.0,74.0,38.0,79.0,160.0,all
AOCS-055-1-7,AOCS-055-1-7,147.0,120.0,17.0,133.0,144.0,78.0,41.0,144.0,181.0,94.0,310.0,138.0,55.0,49.0,63.0,114.0,242.0,105.0,164.0,281.0,41.0,8.0,56.0,64.0,129.0,88.0,14.0,117.0,124.0,87.0,37.0,143.0,122.0,112.0,178.0,198.0,49.0,53.0,75.0,97.0,101.0,136.0,94.0,150.0,38.0,47.0,82.0,118.0,87.0,74.0,19.0,68.0,63.0,58.0,15.0,107.0,137.0,97.0,221.0,106.0,38.0,39.0,42.0,73.0,113.0,72.0,86.0,132.0,32.0,24.0,69.0,62.0,254.0,148.0,26.0,193.0,1586.0,3